In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
pd.options.plotting.backend = "plotly"

In [2]:
NP = pd.read_stata('datos/NelsonPlosserData.dta', index_col='year')
NP.index = NP.index.year

def ZivotAndrewsA(serie, k=8):
    dta = NP[[serie]].dropna()
    dta.rename(columns={serie:'y'}, inplace=True)
    dta['t'] = np.arange(dta.shape[0])
    dta['Ly'] = dta['y'].shift(1)
    dta['Dy'] = dta['y'].diff(1)
    for j in range(1, k+1):
        dta[f'D{j}y'] = dta['Dy'].shift(j)    

    lags = '+'.join(dta.columns[-k:])

    alpha1values = pd.Series(0.0, index=dta.index[12:-12])

    for tau in alpha1values.index:
        dta['DL'] = (dta.index>tau).astype(int)
        modelo = ols('y ~ Ly + t + DL + ' + lags, dta).fit()
        alpha1values[tau] = ((modelo.params - 1)/modelo.bse)['Ly']

    tauhat, tval = alpha1values.idxmin(), alpha1values.min()
    dta['DL'] = (dta.index>tauhat).astype(int)
    modelo = ols('y ~ Ly + t + DL + ' + lags, dta).fit()

    return {r'$\hat{T}_B$':tauhat, r'$\alpha_1$': np.round(modelo.params['Ly'],3), r'$t$': np.round(tval,2)}



seriesA = ['lrgnp', 'lgnp', 'lpcrgnp', 'lip', 'lemp', 'lprgnp', 'lcpi', 'lwg', 'lm']
lags = [8,8,7,8,7,5,2,7,6]
variables = {'lrgnp':'Real GNP',
           'lgnp':'Nominal GNP',
           'lpcrgnp':'Real per capita GNP',
           'lip':'Industrial production',
           'lemp':'Employment',
           'lun':'Unemployment rate',
           'lprgnp':'GNP deflator',
           'lcpi':'Consumer prices',
           'lwg':'Wages',
           'lrwg':'Real wages',
           'lm':'Money stock',
           'lvel':'Velocity',
           'bnd':'Bond yield',
           'lsp500':'Common stock prices'}


temp = pd.DataFrame([ZivotAndrewsA(ser, k) for ser, k in zip(seriesA, lags)], index=seriesA)
temp.rename(index=variables)

,$\hat{T}_B$,$\alpha_1$,$t$
Real GNP,1929,0.267,-5.58
Nominal GNP,1929,0.532,-5.82
Real per capita GNP,1929,0.494,-4.61
Industrial production,1929,0.290,-5.95
Employment,1929,0.651,-4.95
GNP deflator,1929,0.786,-4.12
Consumer prices,1873,0.941,-2.76
Wages,1929,0.660,-5.30
Money stock,1929,0.823,-4.34
